In [1]:
### all imports here - we will make requirements document for these
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from geopy import distance
import folium

In [2]:
### this is to enable the widgets/voila in the notebook - only need once per session
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /Users/a0271775/miniconda3/envs/dva_env/etc/jupyter
    - Validating...
      voila 0.2.16 OK


In [ ]:
### Add all sub-module definitions here
def distance_calc(row, user_input):
    #print('made it')
    #print(row)
    row_loc = (row['Lat Split'], row['Long Split'])
    calc = distance.distance(row_loc, user_input).miles
    return calc
def label(row):
    return_str = ""
    if not 'UNNAMED' in row['Name']:
        return_str = return_str + "Name: " + row['Name']
    return_str = return_str + " Event: " + row['Status']
    if row['Maximum Wind'] != -99:
        return_str = return_str + " Max Wind: " + str(row['Maximum Wind'])
    #print(return_str)
    return return_str

In [ ]:
### read all the csvs
storm_df = pd.read_csv('DataAddressUSA.csv')
zip_df = pd.read_csv('zipcodes.csv')
#fixing the zip codes that start with 00
zip_df['zip'] = zip_df['zip'].apply('{:0>5}'.format)
#zip_df.head()

In [ ]:
### first widget input - zip-code from user
#zip_input = input("What is your zip code (5 digit code and press enter)?") ## actual jupyter version
zip_text = widgets.Text(placeholder='Your 5-digit zip code')
#print(zip_text)

In [ ]:
### call the html page setup here
text_0 = widgets.HTML(value="<hr /><center><h1>Simplified Dashboard for Hurricane Impact!</h1></center>")
text_desc = widgets.HTML(value="<p align=center>Simple interface to identify the impact of hurricanes and floods to your Home and Community. </p>")
text_desc1 = widgets.HTML(value="<p align=center>Start with entering your zipcode! </p> <hr />")
label_0 = widgets.Label('Enter Zip Code here:')

file = open("atl.png", "rb")
im_atl = file.read()
image_0 = widgets.Image(
                    value=im_atl,
                    format='png',
                    width='100'
                )

btn_0 = widgets.Button(
                description='Show the Map',
                tooltip='click here',
                style={'description_width': 'initial'}
            )
out = widgets.Output()
def on_button_clicked(event):
    with out:
        clear_output()
        ### use the input to update tables
        zip_input = zip_text.value
        #print(zip_input)
        lat_user = zip_df[zip_df['zip'] == zip_input]['latitude'].values[0]
        long_user = zip_df[zip_df['zip'] == zip_input]['longitude'].values[0]
        user_input = (lat_user, long_user)
        #print(user_input)
        storm_df['Distance'] = storm_df.apply(lambda row: distance_calc(row, user_input), axis = 1)
        #storm_df_trial = storm_df.loc[1:5, :]
        #storm_df_trial['Label'] = storm_df_trial.apply(lambda row: label(row), axis = 1)
        storm_df['Label'] = storm_df.apply(lambda row: label(row), axis = 1)
        #storm_df.head()
        ### put the plot output here
        map_USA = folium.Map(location=[lat_user, long_user], zoom_start = 8)
        hun_mile_df = storm_df[storm_df['Distance']<=75]
        hun_mile_df.head()
        locations = hun_mile_df[['Lat Split', 'Long Split']]
        locationlist = locations.values.tolist()
        userzip = folium.FeatureGroup(name='Your Zip Code')
        closestzip = folium.FeatureGroup(name='Neighboring Zip Codes w/ Storms')
        userzip.add_to(map_USA)
        closestzip.add_to(map_USA)
        folium.LayerControl().add_to(map_USA)
        for point in range(0, len(locationlist)):
            folium.Marker(locationlist[point], popup=storm_df['Label'][point]).add_to(closestzip)
        folium.Marker([lat_user, long_user], popup='Your Zipcode', icon=folium.Icon(color='red', icon = 'home')).add_to(userzip)
        display(map_USA)
btn_0.on_click(on_button_clicked)


In [ ]:
### need to call the total widget box with components as needed
#vbox_0 = widgets.VBox(children=[image_0], layout=widgets.Layout(align_content='stretch'))
vbox_1 = widgets.VBox([text_0, text_desc, text_desc1, label_0, zip_text, btn_0])
page = widgets.VBox(children=[vbox_1, out],layout=widgets.Layout(width='90%',align_self='center'))
display(page)

In [4]:
### end
#!pip freeze > requirements.txt